In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# !wget https://github.com/skoltech-nlp/detox/releases/download/emnlp2021/filtered_paranmt.zip

In [3]:
# !unzip filtered_paranmt.zip

In [4]:
data = pd.read_csv("../data/filtered.tsv", delimiter="\t")
data.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [5]:
test_data = data.sample(frac=0.2, random_state=42)
data = data.drop(test_data.index)

In [6]:
data = data.drop("Unnamed: 0", axis=1)

In [7]:
test_data.to_csv("../data/test_dataset.tsv", index=False, sep='\t')

In [8]:
import torch
from data.dataset import ToxicityDataset

In [9]:
# !pip install transformers
# !pip install datasets
# !pip install SentencePiece
# !pip install transformers[torch]

In [6]:
import pandas as pd
import torch
from torch.utils.data import random_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, Trainer, TrainingArguments, T5ForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [11]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
dataset = ToxicityDataset(data, tokenizer, max_length=120, task="generation")
train_dataset, test_dataset, validate_dataset = random_split(dataset, [.7, .2, .1])

/opt/conda/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes f

In [12]:
torch.save(test_dataset, f="test_dataset.obj")

In [7]:
# Define the T5 model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name, config=T5Config())

/opt/conda/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes f

In [14]:
training_args = TrainingArguments(
    output_dir="./t5_toxicity_finetuned",
    per_device_train_batch_size=54,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=15000,
    save_steps=15000,
    logging_steps=15000
)

In [15]:
def model_init():
    return T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the trainer and training arguments
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset
)

# Train the model
# trainer.train()


Step,Training Loss,Validation Loss


In [22]:
model_dir = "./saved_model"
trainer.save_model(model_dir)

In [3]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained('./saved_model/')

In [16]:
input = "Shit"
tokens = tokenizer(
    input,
    padding="max_length",
    max_length=50,
    truncation=True,
    return_tensors="pt",
)

out = model.generate(
    input_ids = tokens["input_ids"],
    attention_mask = tokens["attention_mask"],
    max_length= 50,
    num_return_sequences = 1
)

In [18]:
output = [tokenizer.decode(
    gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True
) for gen_id in out]
output

['hell of a job.']